In [26]:
%matplotlib inline
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import graphviz

from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import seaborn as sns

In [2]:
data_boston = load_boston()
print(data_boston.DESCR)
print("Shape:", data_boston.data.shape)

X_train, X_test, y_train, y_test = train_test_split(data_boston.data, data_boston.target, test_size=0.20)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [23]:
def PolynomialRegression(degree=2, linear_model=LinearRegression(), **kwargs):
    return make_pipeline(PolynomialFeatures(degree), linear_model(**kwargs))

def fit_model(X, y):
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits=10, test_size = 0.20, random_state = 0)
 
    #Creating a dictionary for the parameter degree with a range from 1 to 10 and fit_intercept of true or false
    params = {'polynomialfeatures__degree': [1,2,3,4]}

    best_models = []
    for m in [LinearRegression, Lasso, Ridge]:
        grid = GridSearchCV(PolynomialRegression(linear_model=m), param_grid = params, 
                         cv=cv_sets, 
                         scoring='r2',
                         return_train_score=True)
        grid = grid.fit(X, y)
        cv_results = pd.DataFrame(grid.cv_results_)
        print(m)
        print(cv_results[['params', 'mean_fit_time', 'mean_score_time', 'mean_test_score']].sort_values(by='mean_test_score', ascending=False))
        best_models.append(grid.best_estimator_)
    
    return best_models

#Fitting the training data using grid search
models = fit_model(X_train, y_train)

<class 'sklearn.linear_model.base.LinearRegression'>
                              params  mean_fit_time  mean_score_time  \
0  {'polynomialfeatures__degree': 1}       0.002394         0.001396   
1  {'polynomialfeatures__degree': 2}       0.006383         0.002094   
3  {'polynomialfeatures__degree': 4}       0.129254         0.041190   
2  {'polynomialfeatures__degree': 3}       0.053059         0.010173   

   mean_test_score  
0         0.697346  
1        -1.065036  
3      -240.665569  
2     -1493.756884  
<class 'sklearn.linear_model.coordinate_descent.Lasso'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}       0.031814         0.002593   
2  {'polynomialfeatures__degree': 3}       0.182512         0.007879   
0  {'polynomialfeatures__degree': 1}       0.001397         0.000798   
3  {'polynomialfeatures__degree': 4}       0.808739         0.025533   

   mean_test_score  
1         0.808766  
2         0.794251  
0

In [24]:
features_transformed = pd.DataFrame(data=data_boston.data, columns=data_boston.feature_names)

# Applying scaling using MinMaxScaler to the log transformed data set and storing it in a new data frame called 'features_log_minmax_transform'
scaler = MinMaxScaler()
features_transformed[data_boston.feature_names] = scaler.fit_transform(features_transformed)
print(features_transformed.head())

       CRIM    ZN     INDUS  CHAS       NOX        RM       AGE       DIS  \
0  0.000000  0.18  0.067815   0.0  0.314815  0.577505  0.641607  0.269203   
1  0.000236  0.00  0.242302   0.0  0.172840  0.547998  0.782698  0.348962   
2  0.000236  0.00  0.242302   0.0  0.172840  0.694386  0.599382  0.348962   
3  0.000293  0.00  0.063050   0.0  0.150206  0.658555  0.441813  0.448545   
4  0.000705  0.00  0.063050   0.0  0.150206  0.687105  0.528321  0.448545   

        RAD       TAX   PTRATIO         B     LSTAT  
0  0.000000  0.208015  0.287234  1.000000  0.089680  
1  0.043478  0.104962  0.553191  1.000000  0.204470  
2  0.043478  0.104962  0.553191  0.989737  0.063466  
3  0.086957  0.066794  0.648936  0.994276  0.033389  
4  0.086957  0.066794  0.648936  1.000000  0.099338  


In [25]:
X_train_scale, X_test_scale, y_train_scale, y_test_scale = train_test_split(features_transformed, 
                    data_boston.target, test_size = 0.2, random_state = 0)
models = fit_model(X_train_scale, y_train_scale)

<class 'sklearn.linear_model.base.LinearRegression'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}       0.006582         0.001895   
0  {'polynomialfeatures__degree': 1}       0.002792         0.000897   
3  {'polynomialfeatures__degree': 4}       0.116189         0.031616   
2  {'polynomialfeatures__degree': 3}       0.049567         0.006981   

   mean_test_score  
1         0.751177  
0         0.725487  
3       -10.468849  
2       -22.489644  
<class 'sklearn.linear_model.coordinate_descent.Lasso'>
                              params  mean_fit_time  mean_score_time  \
0  {'polynomialfeatures__degree': 1}       0.001596         0.000399   
2  {'polynomialfeatures__degree': 3}       0.019748         0.008078   
3  {'polynomialfeatures__degree': 4}       0.089860         0.030319   
1  {'polynomialfeatures__degree': 2}       0.003690         0.001695   

   mean_test_score  
0         0.292238  
2         0.290059  
3

In [27]:
features_transformed = pd.DataFrame(data=data_boston.data, columns=data_boston.feature_names)

# Applying scaling using MinMaxScaler to the log transformed data set and storing it in a new data frame called 'features_log_minmax_transform'
scaler = StandardScaler()
features_transformed[data_boston.feature_names] = scaler.fit_transform(features_transformed)
print(features_transformed.head())

X_train_scale, X_test_scale, y_train_scale, y_test_scale = train_test_split(features_transformed, 
                    data_boston.target, test_size = 0.2, random_state = 0)
models = fit_model(X_train_scale, y_train_scale)

       CRIM        ZN     INDUS      CHAS       NOX        RM       AGE  \
0 -0.417713  0.284830 -1.287909 -0.272599 -0.144217  0.413672 -0.120013   
1 -0.415269 -0.487722 -0.593381 -0.272599 -0.740262  0.194274  0.367166   
2 -0.415272 -0.487722 -0.593381 -0.272599 -0.740262  1.282714 -0.265812   
3 -0.414680 -0.487722 -1.306878 -0.272599 -0.835284  1.016303 -0.809889   
4 -0.410409 -0.487722 -1.306878 -0.272599 -0.835284  1.228577 -0.511180   

        DIS       RAD       TAX   PTRATIO         B     LSTAT  
0  0.140214 -0.982843 -0.666608 -1.459000  0.441052 -1.075562  
1  0.557160 -0.867883 -0.987329 -0.303094  0.441052 -0.492439  
2  0.557160 -0.867883 -0.987329 -0.303094  0.396427 -1.208727  
3  1.077737 -0.752922 -1.106115  0.113032  0.416163 -1.361517  
4  1.077737 -0.752922 -1.106115  0.113032  0.441052 -1.026501  
<class 'sklearn.linear_model.base.LinearRegression'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}    